In [33]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import re
from IPython.display import display, HTML

In [2]:
# Title of job in search
title = "Data Scientist"

# Location
location = "Los Angeles County"

In [3]:
# link for list of job postings on the left on linkedin
list_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=data%2Bscientist&location=los%2Bangeles%2Bcounty&geoId=&trk=public_jobs_jobs-search-bar_search-submit&start=25"

response = requests.get(list_url)

list_data = response.text
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.findAll("li")

/var/folders/4p/b0r7rdfn5rn__y6drcc8f5m80000gn/T/ipykernel_22196/4017569261.py:8: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  page_jobs = list_soup.findAll("li")


In [4]:
job_id_list = []

for job in page_jobs:
    # find job_ids
    base_card_div = job.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    job_id_list.append(job_id)
    

In [52]:
def clean_description(description):
    if description:
        # Remove unwanted elements
        for element in description.find_all(['span', 'a']):
            element.decompose()

        # Replace bullet points
        for ul in description.find_all('ul'):
            for li in ul.find_all('li'):
                li.insert(0, '-')

        text = description.get_text(separator='\n').strip()
        text = text.replace('\n\n', '')
        text = text.replace('::marker', '-')
        text = text.replace('-\n', '- ')
        text = text.replace('Show less', '').replace('Show more', '')
        return text

In [109]:
job_list = []
for job_id in job_id_list:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_response = requests.get(job_url)
    job_soup = BeautifulSoup(job_response.text, "html.parser")

    # Dictionary for each job posting
    job_post = {}
    
    # company name
    try:
        job_post["company_name"] = job_soup.find("a",{"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"] = None
    
    # company location
    try: 
        job_post["location"] = job_soup.find("span", {"class": "topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job_post["location"] = None
    
    # Role Title
    try:
        job_post["title"] = job_soup.find("h2", {"class":"top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["title"] = None

    # Seniority Level
    try:
        level_header = job_soup.find("h3", {"class": "description__job-criteria-subheader"})
        sen_level = level_header.find_next_sibling("span").text.strip()
        job_post["seniority_level"] = sen_level
    except:
        job_post["seniority_level"] = None
        
    # Employment Type (Fulltime, Part-time, internship)
    try:
        level_header = job_soup.find("h3", {"class": "description__job-criteria-subheader"})
        emp_header = level_header.find_next("h3", {"class": "description__job-criteria-subheader"})
        emp_type = emp_header.find_next_sibling("span").text.strip()
        job_post["employment_type"] = emp_type
        
    except:
        job_post["job_type"] = "Job type not specified"
    
    # Years of Experience
    try:
        desc = job_soup.find("div", {"class": "description__text description__text--rich"}).text
        pattern = r"(\d+\+?\s?years)"
        yoe = re.findall(pattern, desc)
        yoe_string = " ".join(yoe) if yoe else "No YOE found"
        job_post["YOE"] = yoe

    except:
        job_post["YOE"] = None

    # Degree Required

    # Salary
    try:
        job_post["salary"] = job_soup.find("div", {"class": "salary compensation__salary"}).text.strip()
    
    except AttributeError:
        desc = job_soup.find("div", {"class": "description__text description__text--rich"}).text
        pattern = r"\$[\d,]+(?:\.\d{2})?\s?-\s?\$[\d,]+(?:\.\d{2})?"
        salary_range = re.findall(pattern, desc)
        salary_string = " ".join(salary_range) if salary_range else "No salary found"
        job_post["salary"] = salary_string

    # Programming Languages (Python, R, SQL, Java, Scala)

    # ML Skills (TensorFlow, PyTorch, Scikit-learn, Keras)

    # Data Processing & Databases (Pandas, Spark, Hadoop, Snowflake)

    # Cloud Platforms (AWS, GCP, Azure, OCI)

    # Visualization Tools (Tableau, Power BI, D3.js)

    # Big Data Tools (Airflow, Kafka, Kubernetes)
    
    # Description
    try:
        desc = job_soup.find("div", {"class": "description__text description__text--rich"})
        description = clean_description(desc)
        job_post["description"] = description
        
    except:
        job_post["description"] = "No description found"
    

    job_list.append(job_post)

In [110]:
job_list

[{'company_name': 'iHire',
  'location': 'Los Angeles, CA',
  'title': 'Data Scientist / Architect',
  'seniority_level': 'Entry level',
  'employment_type': 'Full-time',
  'YOE': ['6+ years', '10 years'],
  'salary': '$110,000 - $140,000',
  'description': "Career Movement has partnered with iHire to reach top talent for their opening below. Check it out and apply today!\nAre you a \nTableau expert\n with a passion for turning complex data into powerful business insights? our client, a top professional services company, is looking for a \nSenior BI Analytics Engineer\n to join their dynamic team. This role offers the opportunity to drive impactful data initiatives and influence decision-making through innovative analytics.\nWhat You’ll Do\n- Develop and implement data visualizations using Tableau to deliver strategic business insights.\n- Perform complex data analysis to drive informed decision-making.\n- Collaborate with stakeholders to identify opportunities where analytics can add 

In [111]:
test_url = "https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4182723167"
test_response = requests.get(test_url)
test_soup = BeautifulSoup(test_response.text, "html.parser")


header = test_soup.find("h3", {"class": "description__job-criteria-subheader"})
header_name = header.text.strip()
if header_name == "Seniority level":
    seniority_level = header.find_next_sibling("span").text.strip()
    print(seniority_level)
else:
    None

emp_header = header.find_next("h3", {"class": "description__job-criteria-subheader"})


Mid-Senior level


In [112]:
jobs_df = pd.DataFrame(job_list)
jobs_df


,company_name,location,title,seniority_level,employment_type,YOE,salary,description
0,iHire,"Los Angeles, CA",Data Scientist / Architect,Entry level,Full-time,"[6+ years, 10 years]","$110,000 - $140,000",Career Movement has partnered with iHire to re...
1,Vista Applied Solutions Group Inc,"Los Angeles, CA",Data Scientist (Only Locals),Mid-Senior level,Contract,[],No salary found,Looking for only Local Candidates\nMinimum Qua...
2,favorited,"Santa Monica, CA","Data Scientist - T&S, Fraud and CX",Mid-Senior level,Full-time,[3 years],No salary found,Apply to this position if you …\n- Are passion...
3,Keeling Labs,"Los Angeles, CA",Machine Learning Engineer,Entry level,Full-time,[3+ years],"$130,000.00/yr - $165,000.00/yr","Our Mission\nFounded in 2022, Keeling Labs was..."
4,Passes,"Los Angeles, CA",Product Data Scientist,Entry level,Full-time,[6+ years],No salary found,About Passes\nPasses is a leading platform ded...
5,Los Angeles Dodgers,"Los Angeles, CA",Research Engineer,Associate,Full-time,[3 years],"$120,000.00/yr - $130,000.00/yr",Title: Research Engineer\nDepartment: Baseball...
6,Médina,"Beverly Hills, CA",Artificial Intelligence Engineer,Entry level,Full-time,[],No salary found,Company Description\nMédina Market leverages c...
7,The Walt Disney Company,"Santa Monica, CA","Hulu Data Science Graduate Intern, Summer 2025",Mid-Senior level,Internship,[18 years],No salary found,About the Role & Program:\nJoin the Experiment...
8,tvScientific,"Los Angeles, CA",Senior Data Scientist,Mid-Senior level,Full-time,[6+ years],"$162,021.00/yr - $189,000.00/yr",Job Title:\n Senior Data Scientist (Ads)\nLoca...
9,Disney Entertainment,"Santa Monica, CA","Data Science Grad Intern, Summer 2025",Mid-Senior level,Internship,[18 years],No salary found,About The Role And Program\nSupporting Disney ...


In [83]:
iHire_desc = jobs_df.loc[9,'description']
print(iHire_desc)
pattern = r"\$[\d,]+(?:\.\d{2})?\s?-\s?\$[\d,]+(?:\.\d{2})?"
iHire_salary = re.findall(pattern, iHire_desc)
#print(iHire_salary)

About The Role And Program
Supporting Disney Advertising Sales, the Data Science team are subject matter experts in each of the following areas:
- Integrating a broad suite of complex datasets, from digital event logs to sporting event metadata, into carefully curated and reusable data models
- Creating and optimizing audience classification models that enable us to predict and target strategic audience attributes, including brand affinity, purchase behavior, and psychographics
- Publishing audience attributes across an array of ad-serving endpoints
- Working cross-functionally to ensure our data product is aligned with the goals of each strategic sales vertical and the business as a whole
The Data Science team are part of Disney Advertising Sales’ Data & Measurement Sciences group, an outstanding organization made up of Data Scientists, Data Engineers, Data Strategists, Analysts and business domain experts. We collaborate with both internal and external teams to understand how we can 